In [8]:
import requests
import base64

# Define the URL and the payload to send.
url = "http://127.0.0.1:7860"

payload = {
    "prompt": "puppy dog",
    "steps": 5
}

# Send said payload to said URL through the API.
response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)
r = response.json()

# Decode and save the image.
with open("output.png", 'wb') as f:
    f.write(base64.b64decode(r['images'][0]))

In [56]:
import requests
import io
import base64
from PIL import Image

def face_swap(source_image_path, target_image_path, output_path):
    url = "http://127.0.0.1:7860/reactor/image"

    # 이미지 로드 및 base64 인코딩
    with open(source_image_path, "rb") as source_file:
        #source_image = f"data:image/jpeg;base64,{base64.b64encode(source_file.read()).decode('utf-8')}"
        source_image = base64.b64encode(source_file.read()).decode('utf-8')
    
    #print(source_image)
    with open(target_image_path, "rb") as target_file:
        #target_width, target_height = target_img.size
        #target_image = f"data:image/jpeg;base64,{base64.b64encode(target_img.tobytes()).decode('utf-8')}"
        target_image = base64.b64encode(target_file.read()).decode('utf-8')
    #print(target_image)
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    
    payload = {
        "source_image": source_image,
        #"source_image": "",
        "target_image": target_image,
        #"target_image": "",
        "source_faces_index": [0],
        "face_index": [0,1,2,3],
        "upscaler": "None",
        "scale": 1,
        "upscale_visibility": 1,
        "face_restorer": "CodeFormer",
        "restorer_visibility": 1,
        "restore_first": 1,
        "model": "inswapper_128.onnx", #.\models\insightface
        "gender_source": 0,
        "gender_target": 0,
        "save_to_file": 0,
        #"result_file_path": "",
        "device": "GPU",
        "mask_face": 0,
        "select_source": 0,
        #"face_model": "elena.safetensors",
        #"source_folder": "C:/faces",
        #"random_image": 1,
        "upscale_force": 0
    }

    response = requests.post(url=url, headers=headers, json=payload)
    response.raise_for_status()
    
    # 결과 이미지 저장
    result = response.json()
    image_data = base64.b64decode(result['image'])
    image = Image.open(io.BytesIO(image_data))
    image.save(output_path)
    print(f"Face swapped image saved to {output_path}")

# 사용 예시
source_image_path = "./faces/김태희.jpg"  # 소스 얼굴 이미지 경로
target_image_path = "./faces/bk.goldengirls01.jpg"  # 대상 이미지 경로
output_path = "face_swapped_result.png"  # 결과 이미지 저장 경로

face_swap(source_image_path, target_image_path, output_path)

Face swapped image saved to face_swapped_result.png


In [ ]:
base_image = "../face_recognition/test_hanni2.jpg"
target_video = "hanni.mp4"

import cv2
import face_recognition

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file(base_image)
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# 비디오 파일 로드
video_capture = cv2.VideoCapture(target_video)